# Multimodal RAG with the neo4j-genai package

```shell
NEO4J_URI=bolt://localhost:7687
NEO4J_USER=neo4j
NEO4J_PASSWORD=password
NEO4J_DATABASE=neo4j
```

First, let's import everything we need and define some constants

In [1]:
import os
from dotenv import load_dotenv

import neo4j
import ollama

from PIL import Image
from sentence_transformers import SentenceTransformer

from neo4j_genai.retrievers import VectorRetriever, VectorCypherRetriever
from neo4j_genai.types import RetrieverResultItem
from neo4j_genai.embeddings import SentenceTransformerEmbeddings
from neo4j_genai.llm import LLMInterface
from neo4j_genai.llm.types import LLMResponse
from neo4j_genai.generation import GraphRAG

from IPython.display import Image as IPythonImage

/home/estelle/PycharmProjects/llm-playground/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE", "neo4j")

POSTER_INDEX_NAME = "moviePostersEmbedding"
IMAGE_EMBEDDING_MODEL = "clip-ViT-B-32"

Then, we can connect to the Neo4j graph that contains our data (movies):

In [3]:
driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD),
    database=NEO4J_DATABASE,
)

## Search similar images

In this section, we're going to load an impage from disk, embbed it using sentence-transformer, and perform a similarity search based on this embedding vector against all movie's posters in the DB. Note that a vector index has already been created.

In [4]:
model = SentenceTransformer(IMAGE_EMBEDDING_MODEL)

In [5]:
image_path = "./images/Notre-Dame_de_Paris_2013-07-24.jpg"
vector = model.encode(Image.open(image_path)).tolist()
vector[:5]

[-0.0946938693523407,
 -0.041564106941223145,
 -0.31822115182876587,
 -0.030844956636428833,
 0.5638712048530579]

In [6]:
retriever = VectorRetriever(
    driver,
    index_name=POSTER_INDEX_NAME,
)

In [7]:
result = retriever.search(query_vector=vector, top_k=4)
for r in result.items:
    print(r)

content='{\'budget\': 13000000, \'movieId\': \'50912\', \'tmdbId\': \'2266\', \'plotEmbedding\': [-0.0051452359184622765, -0.009240818209946156, 0.0002915928780566901, -0.001550332410261035, 0.001819185446947813, 0.006922564003616571, -0.005228950642049313, 0.001494790893048048, -0.0216499175876379, -0.02407120354473591, -0.0018594329012557864, 0.020902924239635468, -0.006326901726424694, -0.0247022844851017, -0.0011478577507659793, -0.018455877900123596, 0.033717717975378036, -0.0013289713533595204, 0.010979508981108665, -0.013265565037727356, -0.015326234512031078, -0.005663623567670584, -0.0017966468585655093, -0.01378073263913393, -0.011855293065309525, -0.009710908867418766, 0.029905477538704872, -0.027973597869277, 0.011372324079275131, -0.011146938428282738, 0.014308778569102287, -0.01654975861310959, -0.029750926420092583, -0.028772108256816864, -0.021830225363373756, -0.009826821275055408, -0.006594144739210606, -0.008274879306554794, 0.012325383722782135, 0.0084358686581254, 

The results are there but are difficult to understand because we only have one big string. Let's switch to another retriever that gives us more control over the data we want to return:

In [8]:
retriever = VectorCypherRetriever(  # NEW
    driver,
    index_name=POSTER_INDEX_NAME,
    retrieval_query="RETURN node.title as title, node.plot as plot, node.poster as posterUrl, score",  # NEW
)

In [9]:
result = retriever.search(query_vector=vector, top_k=4)
for r in result.items:
    print(r)

content='<Record title="Paris, I Love You (Paris, je t\'aime)" plot=\'Through the neighborhoods of Paris, love is veiled, revealed, imitated, sucked dry, reinvented and awakened.\' posterUrl=\'https://image.tmdb.org/t/p/w440_and_h660_face/w2ymaGlJqgNHjexYQ6h0ByoGzw8.jpg\' score=0.8618369102478027>' metadata=None
content='<Record title=\'Midnight in Paris\' plot="While on a trip to Paris with his fiancée\'s family, a nostalgic screenwriter finds himself mysteriously going back to the 1920s every day at midnight." posterUrl=\'https://image.tmdb.org/t/p/w440_and_h660_face/4wBG5kbfagTQclETblPRRGihk0I.jpg\' score=0.7957582473754883>' metadata=None
content="<Record title='Taxi 2' plot='Police inspector Emilien and his taxi-driver pal Daniel are back, this time on the tail of a group of Japanese yakuza.' posterUrl='https://image.tmdb.org/t/p/w440_and_h660_face/z0js1eYtKxfw4RBxiTukO7q66Rf.jpg' score=0.7839350700378418>" metadata=None
content="<Record title='Koyaanisqatsi (a.k.a. Koyaanisqatsi:

That looks better, but it's still a string... Can we do better? Yes, we can! The 'cypher' retrievers accept a `format_record_function` parameter, that takes as input the neo4j record returned by the retiraval query, and must return a `RetrieverResultItem` with `content` (str) and `metadata` (dict) fields. Let's create a content by concatenating the movie title and its plot, and include the poster URL and other informations in the metadata. In this way, it's straighforward to access the image URL and display it to validate the retriever results:

In [10]:
def format_record_function(record: neo4j.Record) -> RetrieverResultItem:
    return RetrieverResultItem(
        content=f"Movie title: {record.get('title')}, movie plot: {record.get('plot')}",
        metadata={
            "title": record.get('title'),
            "plot": record.get("plot"),
            "poster": record.get("posterUrl"),
            "score": record.get("score"),
        }
    )


retriever = VectorCypherRetriever(
    driver,
    index_name=POSTER_INDEX_NAME,
    retrieval_query="RETURN node.title as title, node.plot as plot, node.poster as posterUrl, score",
    format_record_function=format_record_function,  # NEW
)

result = retriever.search(query_vector=vector, top_k=4)
for r in result.items:
    print(r.content, r.metadata["score"])
    display(IPythonImage(url=r.metadata["poster"]))

Movie title: Paris, I Love You (Paris, je t'aime), movie plot: Through the neighborhoods of Paris, love is veiled, revealed, imitated, sucked dry, reinvented and awakened. 0.8618369102478027


Movie title: Midnight in Paris, movie plot: While on a trip to Paris with his fiancée's family, a nostalgic screenwriter finds himself mysteriously going back to the 1920s every day at midnight. 0.7957582473754883


Movie title: Taxi 2, movie plot: Police inspector Emilien and his taxi-driver pal Daniel are back, this time on the tail of a group of Japanese yakuza. 0.7839350700378418


Movie title: Koyaanisqatsi (a.k.a. Koyaanisqatsi: Life Out of Balance), movie plot: A collection of expertly photographed phenomena with no conventional plot. The footage focuses on nature, humanity and the relationship between them. 0.7798305749893188


## Search Images from its content

So far, we've performed search based on image similarity. But the advantage of multimodal models is to be able to compare text to images. Let's try and find images matching a specific query text.

We will use again the `VectorCypherRetriever`, but this time it needs one more information: how to transform the text to a vector, so that it can be compared to the vector index. This is the role of the `embedded` parameter:

In [11]:
query_text = "Find a movie taking place in Paris and explain the plot."
top_k = 3

In [12]:
retriever = VectorCypherRetriever(
    driver,
    index_name=POSTER_INDEX_NAME,
    retrieval_query="RETURN node.title as title, node.plot as plot, node.poster as posterUrl, score",
    format_record_function=format_record_function,
    embedder=SentenceTransformerEmbeddings(IMAGE_EMBEDDING_MODEL),  # NEW
)


In [13]:
result = retriever.search(query_text=query_text, top_k=top_k)

In [14]:
for r in result.items:
    print(r.content, r.metadata.get("score"))
    display(IPythonImage(url=r.metadata["poster"]))

Movie title: Truth About Charlie, The, movie plot: A young woman in Paris is about to divorce her husband when she discovers... he's dead; and all their money is gone. She meets a mysterious man, who tells her that the money was really his,... 0.6522422432899475


Movie title: Forget Paris, movie plot: Mickey Gordon is a basketball referee who travels to France to bury his father. Ellen Andrews is an American living in Paris who works for the airline he flies on. They meet and fall in ... 0.6489235162734985


Movie title: Paris, I Love You (Paris, je t'aime), movie plot: Through the neighborhoods of Paris, love is veiled, revealed, imitated, sucked dry, reinvented and awakened. 0.6481257081031799


## RAG by searching on images

Now that we are able to find movies that best match the user question by searching their poster embedding, we can move forward and ask an LLM to generate a nice text for the initial user question. To do so, we're going to use the `GraphRAG` class, that requires a `retriever` and an `llm` as input. The llm can be any LLM from `langchain`, for instance `ChatOllama`:

In [15]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3:8b")
rag = GraphRAG(retriever=retriever, llm=llm)
rag_result = rag.search(
    "Find a movie taking place in Paris and explain the plot.", 
    retriever_config={"top_k": top_k},
)
print(rag_result.answer)

Based on your context, I found two movies that take place in Paris:

1. "The Truth About Charlie" (2002) - The plot revolves around a young woman who is about to divorce her husband when she discovers he's dead and all their money is gone. She meets a mysterious man who claims the money was actually his.

2. "Forget Paris" (1995) - Mickey Gordon, a basketball referee, travels to France to bury his father. While there, he meets Ellen Andrews, an American living in Paris who works for the airline he flies on.

3. "Paris, I Love You" (2006) - This film is a collection of nine short stories that take place throughout Paris. The movie explores various aspects of love, including romantic love, familial love, and self-love.


But it doesn't have to be from `langchain`. You can also create your own LLM but subclassing the `LLMInterface`. An example implementation using the `ollama` python package is given below:

In [16]:
class OllamaLLM(LLMInterface):

    def invoke(self, input: str) -> LLMResponse:
        response = ollama.chat(model=self.model_name, messages=[
          {
            'role': 'user',
            'content': input,
          },
        ])
        return LLMResponse(
            content=response["message"]["content"]
        )

In [17]:
rag = GraphRAG(
    retriever=retriever,
    llm=OllamaLLM('llama3:8b')
)

This object is used in a similar way. Optionally, we can ask the rag pipeline to return the context as well, for evaluation or debugging purposes. In this case, the context will be available in a `retriever_result` field of the `rag_result`: 

In [18]:
rag_result = rag.search(
    # "Find a movie with astronauts and explain the plot.", 
    "Find a movie taking place in Paris and explain the plot.", 
    retriever_config={"top_k": top_k},
    return_context=True,
)
print(rag_result.answer)

Let me answer your question!

If I were to recommend a movie that takes place in Paris and explains its plot, I'd suggest "Forget Paris" (1995).

The movie follows Mickey Gordon, a basketball referee who travels to France to bury his father. While there, he meets Ellen Andrews, an American living in Paris who works for the airline he flies on. As they get to know each other, they start falling in love.

The plot revolves around their blossoming romance and Mickey's struggles to adjust to life without his father. The movie is a charming blend of romance, culture, and self-discovery set against the beautiful backdrop of Paris.

If you're looking for another great option, I'd also recommend "Paris, I Love You" (2006), an anthology film that showcases nine short stories exploring love in the city of love. Each segment features a unique cast and director, offering a diverse range of romantic experiences and perspectives.

Let me know if you have any other questions or if there's anything el

In [19]:
[r.content for r in rag_result.retriever_result.items]

["Movie title: Truth About Charlie, The, movie plot: A young woman in Paris is about to divorce her husband when she discovers... he's dead; and all their money is gone. She meets a mysterious man, who tells her that the money was really his,...",
 'Movie title: Forget Paris, movie plot: Mickey Gordon is a basketball referee who travels to France to bury his father. Ellen Andrews is an American living in Paris who works for the airline he flies on. They meet and fall in ...',
 "Movie title: Paris, I Love You (Paris, je t'aime), movie plot: Through the neighborhoods of Paris, love is veiled, revealed, imitated, sucked dry, reinvented and awakened."]